<a href="https://colab.research.google.com/github/NeonKazuha/Chatbot-/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!kaggle datasets download -d niraliivaghani/chatbot-dataset

Dataset URL: https://www.kaggle.com/datasets/niraliivaghani/chatbot-dataset
License(s): DbCL-1.0
chatbot-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip chatbot-dataset.zip

Archive:  chatbot-dataset.zip
replace intents.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: intents.json            


In [60]:
import pandas as pd
import numpy as np
import json
import random
random.seed(42)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, Embedding, SimpleRNN, LSTM, GRU, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [7]:
with open('intents.json') as f:
    data = json.load(f)

type(data)

dict

In [8]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

In [9]:
tag = data['intents'][0]['tag']
tag

'greeting'

In [10]:
input = data['intents'][0]['patterns']
input

['Hi',
 'How are you?',
 'Is anyone there?',
 'Hello',
 'Good day',
 "What's up",
 'how are ya',
 'heyy',
 'whatsup',
 '??? ??? ??']

In [11]:
len(data['intents'])

38

In [12]:
mapping = {}
for i in range(len(data['intents'])):
    for j in range(len(data['intents'][i]['patterns'])):
        # print(data['intents'][i]['patterns'][j])
        mapping[data['intents'][i]['patterns'][j]] = data['intents'][i]['tag']
    break
mapping

{'Hi': 'greeting',
 'How are you?': 'greeting',
 'Is anyone there?': 'greeting',
 'Hello': 'greeting',
 'Good day': 'greeting',
 "What's up": 'greeting',
 'how are ya': 'greeting',
 'heyy': 'greeting',
 'whatsup': 'greeting',
 '??? ??? ??': 'greeting'}

In [13]:
mapping = {}
for i in range(len(data['intents'])):
    for j in range(len(data['intents'][i]['patterns'])):
        # print(data['intents'][i]['patterns'][j])
        mapping[data['intents'][i]['patterns'][j]] = data['intents'][i]['tag']
mapping

{'Hi': 'greeting',
 'How are you?': 'greeting',
 'Is anyone there?': 'greeting',
 'Hello': 'greeting',
 'Good day': 'greeting',
 "What's up": 'greeting',
 'how are ya': 'greeting',
 'heyy': 'greeting',
 'whatsup': 'greeting',
 '??? ??? ??': 'greeting',
 'cya': 'goodbye',
 'see you': 'goodbye',
 'bye bye': 'goodbye',
 'See you later': 'goodbye',
 'Goodbye': 'goodbye',
 'I am Leaving': 'goodbye',
 'Bye': 'goodbye',
 'Have a Good day': 'goodbye',
 'talk to you later': 'goodbye',
 'ttyl': 'goodbye',
 'i got to go': 'goodbye',
 'gtg': 'goodbye',
 'what is the name of your developers': 'creator',
 'what is the name of your creators': 'creator',
 'what is the name of the developers': 'creator',
 'what is the name of the creators': 'creator',
 'who created you': 'creator',
 'your developers': 'creator',
 'your creators': 'creator',
 'who are your developers': 'creator',
 'developers': 'creator',
 'you are made by': 'creator',
 'you are made by whom': 'creator',
 'who create you': 'creator',
 '

In [14]:
mapping['Hi']

'greeting'

In [15]:
items = list(mapping.items())
random.shuffle(items)
mapping = dict(items)
mapping

{'good job': 'salutaion',
 'hostel facility': 'hostel',
 'Thank You': 'salutaion',
 'scholarship chemical': 'scholarship',
 'Food': 'canteen',
 'distance between college and hostel': 'hostel',
 'events organised': 'event',
 'please provide sports and games information': 'sports',
 'fees for AC room': 'fees',
 'college is located at': 'location',
 'Holiday in these year': 'vacation',
 'what is the fees of each year': 'fees',
 'Can i get your phone number': 'number',
 'how far is hostel from college': 'hostel',
 'asshole': 'swear',
 'sports facilities': 'sports',
 'why i should use you': 'task',
 'how to get in hostel': 'hostel',
 'Where is HOD': 'ithod',
 'whats your name?': 'name',
 'branches?': 'course',
 'list of events conducted in college': 'event',
 'is there any canteen': 'canteen',
 'phone number': 'number',
 'it hod': 'ithod',
 'Cafetaria': 'canteen',
 'scholarship for computer engineering': 'scholarship',
 'Is anyone there?': 'greeting',
 'What is average package': 'placement'

In [16]:
X, y = [], []
for i, j in mapping.items():
    X.append(i)
    y.append(j)
X, y

(['good job',
  'hostel facility',
  'Thank You',
  'scholarship chemical',
  'Food',
  'distance between college and hostel',
  'events organised',
  'please provide sports and games information',
  'fees for AC room',
  'college is located at',
  'Holiday in these year',
  'what is the fees of each year',
  'Can i get your phone number',
  'how far is hostel from college',
  'asshole',
  'sports facilities',
  'why i should use you',
  'how to get in hostel',
  'Where is HOD',
  'whats your name?',
  'branches?',
  'list of events conducted in college',
  'is there any canteen',
  'phone number',
  'it hod',
  'Cafetaria',
  'scholarship for computer engineering',
  'Is anyone there?',
  'What is average package',
  'your name',
  'scholarship engineering',
  'it',
  'book facility',
  'what are your hours',
  'creators',
  'when is semester',
  'floors in college',
  'seats',
  'contact info',
  'Computer engineering?',
  'food menu',
  'About placement',
  'who are your developers'

In [17]:
y = pd.DataFrame(y)
label_mapping = dict(enumerate(y[0].astype('category').cat.categories))
label_mapping

{0: 'admission',
 1: 'canteen',
 2: 'college intake',
 3: 'committee',
 4: 'computerhod',
 5: 'course',
 6: 'creator',
 7: 'document',
 8: 'event',
 9: 'extchod',
 10: 'facilities',
 11: 'fees',
 12: 'floors',
 13: 'goodbye',
 14: 'greeting',
 15: 'hod',
 16: 'hostel',
 17: 'hours',
 18: 'infrastructure',
 19: 'ithod',
 20: 'library',
 21: 'location',
 22: 'menu',
 23: 'name',
 24: 'number',
 25: 'placement',
 26: 'principal',
 27: 'ragging',
 28: 'random',
 29: 'salutaion',
 30: 'scholarship',
 31: 'sem',
 32: 'sports',
 33: 'swear',
 34: 'syllabus',
 35: 'task',
 36: 'uniform',
 37: 'vacation'}

In [18]:
y['target'] = y[0].astype('category').cat.codes
y

,0,target
0,salutaion,29
1,hostel,16
2,salutaion,29
3,scholarship,30
4,canteen,1
...,...,...
392,location,21
393,task,35
394,goodbye,13
395,hours,17


In [19]:
len(X)

397

In [20]:
def split_data(X, y, ratio):
    n = len(X)
    X_train = X[:int(n*ratio)]
    X_test = X[int(n*ratio):]
    y_train = y[:int(n*ratio)]
    y_test = y[int(n*ratio):]
    return X_train, X_test, y_train, y_test

In [30]:
X_train, X_test, y_train, y_test = split_data(X, y['target'], 0.8)
X_train, y_train

(['good job',
  'hostel facility',
  'Thank You',
  'scholarship chemical',
  'Food',
  'distance between college and hostel',
  'events organised',
  'please provide sports and games information',
  'fees for AC room',
  'college is located at',
  'Holiday in these year',
  'what is the fees of each year',
  'Can i get your phone number',
  'how far is hostel from college',
  'asshole',
  'sports facilities',
  'why i should use you',
  'how to get in hostel',
  'Where is HOD',
  'whats your name?',
  'branches?',
  'list of events conducted in college',
  'is there any canteen',
  'phone number',
  'it hod',
  'Cafetaria',
  'scholarship for computer engineering',
  'Is anyone there?',
  'What is average package',
  'your name',
  'scholarship engineering',
  'it',
  'book facility',
  'what are your hours',
  'creators',
  'when is semester',
  'floors in college',
  'seats',
  'contact info',
  'Computer engineering?',
  'food menu',
  'About placement',
  'who are your developers'

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [23]:
Xtrain = tokenizer.texts_to_sequences(X_train)
Xtest = tokenizer.texts_to_sequences(X_test)
Xtrain = pad_sequences(Xtrain)
Xtrain.shape

(317, 9)

In [24]:
dim = Xtrain.shape[1]
dim

9

In [25]:
Xtest = pad_sequences(Xtest, maxlen=dim)

In [26]:
word2idx = tokenizer.word_index
V = len(word2idx)
V = V + 1
V, word2idx

(280,
 {'is': 1,
  'college': 2,
  'what': 3,
  'the': 4,
  'you': 5,
  'of': 6,
  'in': 7,
  'for': 8,
  'scholarship': 9,
  'are': 10,
  'hostel': 11,
  'how': 12,
  'name': 13,
  'i': 14,
  'fees': 15,
  'your': 16,
  'hod': 17,
  'can': 18,
  'to': 19,
  'where': 20,
  'about': 21,
  'who': 22,
  'admission': 23,
  'there': 24,
  'engineering': 25,
  'when': 26,
  'courses': 27,
  'do': 28,
  'have': 29,
  'library': 30,
  'me': 31,
  'documents': 32,
  'events': 33,
  'number': 34,
  'facilities': 35,
  'list': 36,
  'canteen': 37,
  'it': 38,
  'facility': 39,
  'information': 40,
  'any': 41,
  'computer': 42,
  'semester': 43,
  'uni': 44,
  'available': 45,
  'fee': 46,
  'ragging': 47,
  'sports': 48,
  'year': 49,
  'address': 50,
  'tell': 51,
  'exam': 52,
  'ac': 53,
  'room': 54,
  'at': 55,
  'does': 56,
  'timing': 57,
  'uniform': 58,
  'principal': 59,
  'food': 60,
  'branches': 61,
  'contact': 62,
  'developers': 63,
  'committee': 64,
  'offered': 65,
  'holidays

In [31]:
ytrain = pd.DataFrame(y_train)
ytest = pd.DataFrame(y_test)

In [54]:
input = Input(shape = (dim,))
x = Embedding(V, 50)(input)
x = GRU((20), return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
output = Dense(len(label_mapping), activation = 'softmax')(x)
model = Model(input, output, name='GRU')

In [55]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = Adam(learning_rate = 0.01),
    metrics = ['accuracy']
)

In [56]:
model.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding_6 (Embedding)     (None, 9, 50)             14000     
                                                                 
 gru_6 (GRU)                 (None, 9, 20)             4320      
                                                                 
 global_max_pooling1d_6 (Gl  (None, 20)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 38)                798       
                                                                 
Total params: 19118 (74.68 KB)
Trainable params: 19118 (74.68 KB)
Non-trainable params: 0 (0.00 Byte)
___________________________

In [57]:
result = model.fit(
    Xtrain, ytrain,
    validation_data = (Xtest, ytest),
    epochs = 30,
    batch_size = 128
)

Epoch 1/30
3/3 [==============================] - 3s 283ms/step - loss: 3.6283 - accuracy: 0.0473 - val_loss: 3.5908 - val_accuracy: 0.0750
Epoch 2/30
3/3 [==============================] - 0s 35ms/step - loss: 3.5582 - accuracy: 0.0852 - val_loss: 3.5207 - val_accuracy: 0.1000
Epoch 3/30
3/3 [==============================] - 0s 30ms/step - loss: 3.4579 - accuracy: 0.1167 - val_loss: 3.4371 - val_accuracy: 0.0875
Epoch 4/30
3/3 [==============================] - 0s 37ms/step - loss: 3.3174 - accuracy: 0.0946 - val_loss: 3.3489 - val_accuracy: 0.1125
Epoch 5/30
3/3 [==============================] - 0s 32ms/step - loss: 3.1490 - accuracy: 0.1388 - val_loss: 3.2209 - val_accuracy: 0.2000
Epoch 6/30
3/3 [==============================] - 0s 32ms/step - loss: 2.9350 - accuracy: 0.3028 - val_loss: 3.0479 - val_accuracy: 0.3250
Epoch 7/30
3/3 [==============================] - 0s 32ms/step - loss: 2.6743 - accuracy: 0.4196 - val_loss: 2.8587 - val_accuracy: 0.3625
Epoch 8/30
3/3 [==========

In [58]:
import plotly.express as px
fig = px.line(result.history, y=['loss', 'val_loss'])
fig.show()

In [59]:
import plotly.express as px
fig = px.line(result.history, y=['accuracy', 'val_accuracy'])
fig.show()

In [61]:
def predict(text):
    pred = model.predict(pad_sequences(tokenizer.texts_to_sequences([text]), maxlen = dim))
    prediction = np.argmax(pred)
    predicted_label = label_mapping[prediction]
    return predicted_label

In [62]:
predict('How are you')

1/1 [==============================] - 1s 723ms/step


'greeting'

In [63]:
def answer(text):
    predicted_label = predict(text)
    for i in range(len(data['intents'])):
        if data['intents'][i]['tag'] == predicted_label:
            return random.choice(data['intents'][i]['responses'])

In [64]:
answer('How are you')

1/1 [==============================] - 0s 35ms/step


'Hi there, how can I help?'

In [66]:
answer('what is the syllabus for IT branch of your college')

1/1 [==============================] - 0s 22ms/step


'Timetable provide direct to the students OR To know about syllabus visit <a target="_blank" href="TIMETABLE LINK"> here</a>'